# Data Partitioning

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
random_state = 123

In [3]:
print("Start Data Partitioning")

Start Data Partitioning


In [4]:
# read cleaned data
df = pd.read_csv(r'../raw_data/label_data.csv')
df.head()

,Case_ID,Patient_ID,Date_MRI,ID_MRI_Machine,Entry_date,Operation_date,Adenoma_size,Diagnosis,Category,Patient_age,...,Pre_OP_hormone_intakt,Post_OP_hormone_cortico,Post_OP_hormone_gonado,Post_OP_hormone_somato,Post_OP_hormone_ADH,Post_OP_hormone_thyreo,Post_OP_hormone_hyperprolaktin,Post_OP_hormone_keine,Post_OP_hormone_intakt,Label_Quality
0,41835743,300146159,2023-05-11 09:00:00,MRI3,2021-09-01,2021-09-17,makro,inaktiv (gonado),non-prolaktinom,57,...,0,0,1,0,0,0,0,0,0,NaN
1,41708812,762512,2023-05-06 08:12:00,MRI3,2018-09-01,2018-09-19,makro,gh,non-prolaktinom,66,...,0,0,0,0,0,1,0,0,0,NaN
2,41892695,365189,2023-05-05 14:19:00,MRI3,NaN,NaN,NaN,NaN,NaN,32,...,0,0,0,0,0,0,0,0,0,keine daten
3,41725372,543641,2023-05-05 07:54:00,MRI2,2006-01-01,2009-06-04,mikro,acth,non-prolaktinom,39,...,0,0,0,0,0,0,0,1,0,NaN
4,41843364,300302329,2023-05-02 12:04:00,MRI3,2021-10-01,2022-01-12,makro,inaktiv,non-prolaktinom,56,...,1,0,0,0,0,0,0,0,1,NaN


In [6]:
# read cleaned labor data
labor_data = pd.read_csv(r'../raw_data/labor_data_preprocessed.csv')
labor_data.head()

,FALL_NR,PATIENT_NR,Analyse,Analyse-ID,Auftraggeber,Datum_Resultat,Resultat,Einheit,Normwert,Fallart,Warnung
0,41505731,23613,LH (luteinisierendes Hormon),LH,ENDOKRINOLOGIE,2023-09-08,13.9,U/l,1.50-9.30,ambulant,ausserhalb Norm
1,41919082,300065854,FSH (Follikel-stimulierendes Hormon),FSH,PÄD.ENDOKRINO/DIABET.,2023-07-10,2.25,U/l,1.40-18.1,ambulant,NaN
2,41643796,628910,LH (luteinisierendes Hormon),LH,GYN.ENDOKR.,2023-01-16,3.99,U/l,zyklusabhängig,ambulant,NaN
3,41424235,444742,FSH (Follikel-stimulierendes Hormon),FSH,PÄD.ENDOKRINO/DIABET.,2023-03-31,3.00,U/l,1.40-18.1,ambulant,NaN
4,41777042,500857,LH (luteinisierendes Hormon),LH,ENDOKRINOLOGIE,2023-03-07,22.3,U/l,zyklusabhängig,ambulant,NaN


## Train/Test Split Strategy
We will Split the Data into a Train- and Testset. 
We are splitting each patient fully into either the train or the test set to avoid data leakage.
This is ensured checking for Patient ID duplicates (assert statement beneath) in the Dataframe. If each row only corresponds to one patient we can savely split the dataframe.


Also we are gonna stratify the split on the binary labels to ensure that the train- and testset include about the same of each class.

In [4]:
# drop patients without a label
df= df[~df["Category"].isna()]

In [5]:
# Patient ID Duplicate Check
assert len(df[df["Patient_ID"].duplicated()]) == 0

In [6]:
# split columns into features and labels
X = df.drop(columns=["Category"])
y = df["Category"]
# stratified train/test split on labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

In [7]:
print("Total Dataframe Train rows:", len(X_train))
print("Total Dataframe Test rows:", len(X_test))

Total Dataframe Train rows: 138
Total Dataframe Test rows: 35


In [8]:
label_diff = ((y_train.value_counts(normalize=True) - y_test.value_counts(normalize=True)) *100).iloc[1]

In [9]:
print(f"Label Distribution relative Difference between Train- and Testset:\n","±",np.round(label_diff,3),"%")

Label Distribution relative Difference between Train- and Testset:
 ± 0.166 %


In [10]:
# Create dataframes for training and test data
train_data = pd.DataFrame(X_train)
train_data['Category'] = y_train

test_data = pd.DataFrame(X_test)
test_data['Category'] = y_test

In [11]:
df_more_data = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='w duplicates')

In [12]:
df_more_data =df_more_data[["Fall Nr.","PID"]]
df_more_data= df_more_data.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",})
df_more_data

,Case_ID,Patient_ID
0,0041835743,300146159
1,0041708812,762512
2,0041892695,365189
3,0041725372,543641
4,0041843364,300302329
...,...,...
1193,0004213315,112374
1194,0004211936,153807
1195,0004180070,719666
1196,0004139115,313269


In [13]:
train_data_merged = train_data.merge(df_more_data,how='left',on='Patient_ID')
test_data_merged = test_data.merge(df_more_data,how='left',on='Patient_ID')

In [14]:
train_data_merged = train_data_merged.drop(columns="Case_ID_x")
train_data_merged= train_data_merged.rename(columns={"Case_ID_y": "Case_ID"})

test_data_merged = test_data_merged.drop(columns="Case_ID_x")
test_data_merged= test_data_merged.rename(columns={"Case_ID_y": "Case_ID"})

In [15]:
# Save the training and test data to CSV files
train_data_merged.to_csv(r'../data/train_data.csv', index=False)
test_data_merged.to_csv(r'../data/test_data.csv', index=False)

In [16]:
assert len(train_data_merged.Patient_ID.unique()) == len(train_data.Patient_ID.unique())
assert len(test_data_merged.Patient_ID.unique()) == len(test_data.Patient_ID.unique())

In [17]:
print("End Data Partitioning")

End Data Partitioning
